# Spark Consumer

TODO: find how to make output to 2 places : save json to hdfs and table to hive

how come sometimes there are 2 tweets in one output, like:


+----+--------------------+
| key|               value|
+----+--------------------+
|null|"@SenWarren Need ...|
|null|"RT @TrinityResis...|
+----+--------------------+


In [ ]:
from __future__ import print_function

import sys
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


In [ ]:
topic1 = 'TweeterArchive'

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell' 

In [ ]:
spark = SparkSession\
    .builder\
    .appName("StructuredNetworkWordCount")\
    .getOrCreate()

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic1) \
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:

query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()
query.awaitTermination()

In [ ]:
words = df.select(
   explode(
       split(df.value, " ")
   ).alias("word")
)

# Generate running word count
#wordCounts = words.groupBy("word").count()

In [ ]:
# query = wordCounts \
#     .writeStream \
#     .outputMode("complete") \
#     .format("console") \
#     .start()
# query.awaitTermination()

In [ ]:
query = word \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()